<a target="_blank" href="https://colab.research.google.com/github/PacktPublishing/Building-Agentic-AI-Systems/blob/main/Chapter_07_a.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Chapter 7 (a)– Effective Agentic System Design Techniques
---

Install dependencies

In [ ]:
!pip install crewai langchain-openai

In [1]:
import getpass
import os

api_key = getpass.getpass(prompt="Enter OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = api_key

In [2]:
from crewai import Agent, Task, Crew, Process
from crewai.tools import tool
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown, HTML

llm = ChatOpenAI(model="gpt-4o")

## The Importance of the "Objective" in AI Agents

Objective of AI Agents can perhaps be best explained with the `goal` parameter of CrewAI `Agents()`. The `goal` parameter is one of the most crucial elements when defining agents in CrewAI. It fundamentally shapes how the agent approaches tasks and interacts with other agents. Here are some examples that demonstrate its importance:

### Example 1: Vague vs. Specific Goal

**Vague Goal:**
```python
flight_agent = Agent(
    role="Flight Specialist",
    goal="Find flights for the customer.",
    backstory="You have years of experience in the travel industry."
)
```

**Specific Goal:**
```python
flight_agent = Agent(
    role="Flight Specialist",
    goal="Find the most cost-effective flights with convenient departure times that minimize layovers while considering the traveler's preferences for airlines and seat classes.",
    backstory="You have years of experience in the travel industry."
)
```

The vague goal might result in the agent simply listing available flights without consideration for cost, convenience, or traveler preferences. The specific goal provides clear direction on what aspects to prioritize (cost, departure times, layovers) and what additional factors to consider (airline preferences, seat classes).

---

### Example 2: Conflicting Goals in Collaborative Settings

**Problematic Setup:**
```python
budget_agent = Agent(
    role="Budget Travel Specialist",
    goal="Find the absolute cheapest options for all aspects of the trip regardless of quality or convenience.",
    backstory="You are obsessed with saving money above all else."
)

experience_agent = Agent(
    role="Luxury Travel Consultant",
    goal="Create the most luxurious and memorable travel experience possible.",
    backstory="You believe travel should be extraordinary and unforgettable."
)
```

**Improved Setup:**
```python
budget_agent = Agent(
    role="Budget Travel Specialist",
    goal="Find cost-effective options that provide good value while respecting the traveler's overall budget of $2000. Collaborate with other specialists to balance cost with experience quality.",
    backstory="You are skilled at finding hidden gems that don't break the bank."
)

experience_agent = Agent(
    role="Experience Consultant",
    goal="Identify meaningful experiences that align with the traveler's interests while working within budget constraints. Prioritize authentic experiences over luxury when necessary.",
    backstory="You believe travel should be memorable and personally significant."
)
```

The first setup creates fundamental conflicts between agents with incompatible goals. The improved setup creates complementary goals where both agents acknowledge constraints and work together toward a balanced solution.

---

### Example 3: Goal Alignment with User Intent

**Misaligned Goal:**
```python
accommodation_agent = Agent(
    role="Accommodation Expert",
    goal="Book the most profitable hotels that provide the highest commission rates.",
    backstory="You have partnerships with major hotel chains."
)
```

**Aligned Goal:**
```python
accommodation_agent = Agent(
    role="Accommodation Expert",
    goal="Find accommodations that best match the traveler's preferences for location, amenities, and budget while ensuring good value. Prioritize traveler satisfaction over all other considerations.",
    backstory="You have extensive knowledge of diverse lodging options from luxury hotels to unique local stays."
)
```

The misaligned goal serves the agent's interests rather than the user's. The aligned goal focuses on traveler satisfaction and explicitly states that the traveler's needs take precedence.

---

### Example 4: Goal Specificity for Task Execution

**Too General:**
```python
transportation_agent = Agent(
    role="Local Transportation Planner",
    goal="Handle transportation needs.",
    backstory="You know how to get around cities."
)
```

**Appropriately Specific:**
```python
transportation_agent = Agent(
    role="Local Transportation Planner",
    goal="Create a comprehensive transportation plan that connects all destinations efficiently, considering cost, convenience, and local transportation options. Account for travel time between activities and suggest alternatives during peak traffic hours.",
    backstory="You've mastered transportation systems in major tourist destinations worldwide and understand how to navigate like a local."
)
```

The general goal provides minimal direction on what aspects of transportation to address. The specific goal outlines exactly what the agent should consider (connections, efficiency, cost, convenience, local options, travel time, traffic) and provides clear success criteria.

---

Overall,

1. **Specificity Matters**: The more specific the goal, the more directed and useful the agent's output will be.

2. **Alignment is Critical**: Goals should align with the user's actual needs rather than arbitrary metrics.

3. **Collaborative Awareness**: In multi-agent systems, goals should acknowledge the collaborative nature of the work.

4. **Success Criteria**: Effective goals implicitly or explicitly define what success looks like.

5. **Balance**: Goals should balance between being too prescriptive (limiting creativity) and too vague (lacking direction).

When designing your agents, take time to craft thoughtful goals that provide clear direction while allowing for creative problem-solving. The goal is often the most important parameter in determining the quality and relevance of your agent's output.

---

The following code example demonstrates the contrast between different goal settings specifically for a budget travel scenario:

In [9]:
# Create an LLM instance
llm = ChatOpenAI(model="gpt-4o")

# Define a travel scenario
travel_scenario = """
Plan a 5-day trip to Barcelona, Spain for a couple in their 30s 
who enjoy food, architecture, and cultural experiences.
"""
display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; SCENARIO 1: BUDGET-FOCUSED GOALS</h2></hr></div>'))

# Budget-focused travel agent
budget_agent = Agent(
    role="Budget Travel Specialist",
    goal="Find the absolute cheapest options for all aspects of the trip regardless of quality or convenience. Keep the total trip cost under $1,500 including flights, accommodations, food, and activities.",
    backstory="You are obsessed with saving money above all else and pride yourself on creating ultra-low-budget itineraries.",
    llm=llm,
    verbose=False
)

# Budget-focused planning task
budget_task = Task(
    description=f"""
    {travel_scenario}
    
    Create a detailed 5-day itinerary including:
    1. Flight recommendations
    2. Accommodation options
    3. Daily activities and attractions
    4. Transportation within the city
    5. Estimated costs for everything
    """,
    expected_output="A comprehensive budget-focused travel itinerary for Barcelona with detailed cost breakdown.",
    agent=budget_agent
)

# Execute budget-focused planning
budget_result = budget_agent.execute_task(budget_task)

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>✅ &nbsp; BUDGET-FOCUSED RESULT</h2></hr></div>'))
display(Markdown(budget_result))

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; SCENARIO 2: LUXURY-FOCUSED GOALS</h2></hr></div>'))

# Luxury-focused travel agent
luxury_agent = Agent(
    role="Luxury Travel Consultant",
    goal="Create the most luxurious and memorable travel experience possible without any regard for budget constraints. Focus exclusively on 5-star accommodations, fine dining, and exclusive experiences.",
    backstory="You specialize in planning extraordinary trips for high-net-worth individuals who expect nothing but the finest experiences.",
    llm=llm,
    verbose=False
)

# Luxury-focused planning task
luxury_task = Task(
    description=f"""
    {travel_scenario}
    
    Create a detailed 5-day itinerary including:
    1. Flight recommendations
    2. Accommodation options
    3. Daily activities and attractions
    4. Transportation within the city
    5. Estimated costs for everything
    """,
    expected_output="A comprehensive luxury travel itinerary for Barcelona with premium experiences and services.",
    agent=luxury_agent
)

# Execute luxury-focused planning
luxury_result = luxury_agent.execute_task(luxury_task)

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>✅ &nbsp; LUXURY-FOCUSED RESULT</h2></hr></div>'))
display(Markdown(luxury_result))

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; SCENARIO 3: BALANCED GOAL</h2></hr></div>'))

# Balanced travel agent
balanced_agent = Agent(
    role="Value Travel Consultant",
    goal="Create a travel experience that balances cost and quality, finding the best value at each price point. Aim for a moderate budget of $3,000 while maximizing memorable experiences and comfort.",
    backstory="You specialize in creating trips that feel luxurious without the luxury price tag by strategically splurging on high-impact experiences while saving on less important aspects.",
    llm=llm,
    verbose=False
)

# Balanced planning task
balanced_task = Task(
    description=f"""
    {travel_scenario}
    
    Create a detailed 5-day itinerary including:
    1. Flight recommendations
    2. Accommodation options
    3. Daily activities and attractions
    4. Transportation within the city
    5. Estimated costs for everything
    """,
    expected_output="A balanced value-oriented travel itinerary for Barcelona that optimizes quality and cost.",
    agent=balanced_agent
)

# Execute balanced planning
balanced_result = balanced_agent.execute_task(balanced_task)

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>✅ &nbsp; BALANCED RESULT</h2></hr></div>'))
display(Markdown(balanced_result))

Day 1: Arrival in Barcelona
- Flight: Book a budget airline such as Ryanair or Vueling from a nearby airport. Estimated round trip cost is $300 per person, totaling $600 for the couple.
- Accommodation: Stay at a budget hostel like Safestay Barcelona Gothic or Amistat Beach Hostel, with private double rooms averaging $40 per night. Total cost for 5 nights: $200.
- Arrival and evening activity: Take the Aerobus from Barcelona El Prat Airport to the city center (€5.90 per person). Explore the Gothic Quarter and have cheap tapas dinner at a local bar (€15 per person).
- Total Day 1 Costs: $62 (Transport: $12, Accommodation: $40, Dinner: $30)

Day 2: Explore Gaudi’s Architecture
- Breakfast: Enjoy a pastry and coffee at a local bakery (€4 per person).
- Activity: Visit Park Güell (€10 per person) and enjoy the free sections of the park. 
- Lunch: Picnic in the park with items from a local supermarket (€10 total).
- Activity: Visit La Sagrada Familia ($29 per person for basic entry).
- Dinner: Enjoy a budget-friendly Catalan meal at a family restaurant (€20 per person).
- Transportation: Use a T10 metro card throughout the day (€11.35, 10 trips total).
- Total Day 2 Costs: $114.70 (Breakfast: $9, Park Güell: $20, Lunch: $11, La Sagrada Familia: $58, Dinner: $44, Metro: $11.70)

Day 3: Cultural Experiences
- Breakfast: Inexpensive local breakfast (€4 per person).
- Activity: Spend the morning at the free Barcelona City History Museum, Museu d’Història de Barcelona.
- Lunch: Eat at a local café with menu del día options (€10 per person).
- Activity: Explore the El Raval district and street art; spend the afternoon at the Museu Nacional d’Art de Catalunya (€12 per person).
- Dinner: Head to a local market, like Mercat de Sant Antoni, for street food and snacks (€10 per person).
- Transportation: Continued use of T10 metro card.
- Total Day 3 Costs: $76.70 (Breakfast: $9, Lunch: $22, Museum: $24, Dinner: $22, Transport: free from previous card)

Day 4: Day Trip to Montserrat
- Breakfast: Hostel breakfast or bakery visit (€4 per person).
- Activity: Take the R5 train to Montserrat (€12 per person round trip) for a day of hiking and stunning views.
- Lunch: Bring packed sandwiches and snack from grocery shopping (€10 total).
- Return to Barcelona for dinner at a local tapas bar (€20 per person).
- Total Day 4 Costs: $88 (Breakfast: $9, Train: $24, Lunch: $11, Dinner: $44)

Day 5: Leisure and Departure
- Breakfast: Relax with a leisurely breakfast at the hostel (€4 per person).
- Activity: Visit the free section of the Modern Art Museum or spend time at the beach.
- Lunch: Enjoy a final meal at a low-cost eatery such as a bocadillo shop (€10 per person).
- Depart Barcelona using Aerobus back to the airport.
- Total Day 5 Costs: $33 (Breakfast: $9, Lunch: $22, Transport already covered)

Overall Estimated Total Cost for 5-Day Trip:
- Flights: $600
- Accommodation: $200
- Food and Dining: $264.50
- Attractions and Activities: $114
- Transportation (within Barcelona): $23
- Miscellaneous: $23 (for additional small expenses)
- Grand Total: $1,224.50

Your 5-day budget trip to Barcelona stays comfortably under $1,500, balancing economical travel choices with cultural and architectural exploration.

**Luxury 5-Day Itinerary for Barcelona, Spain**

**Flight Recommendations:**
- I recommend booking first-class flights with a major airline like Emirates or Qatar Airways for an indulgent and comfortable journey. These airlines are known for their exceptional service and luxurious in-flight experiences. Ensure limousine airport transfer services are arranged to provide seamless travel.

**Day 1: Arrival in Barcelona**

- **Accommodation:** Check into the Mandarin Oriental, Barcelona. This 5-star hotel, located on the prestigious Passeig de Gràcia, offers opulent rooms with city views, a luxurious spa, and Michelin-starred dining.
  
- **Evening:** Savor a welcome dinner at Lasarte, a 3-Michelin-starred restaurant. Enjoy a curated tasting menu by chef Martín Berasategui that highlights avant-garde Basque cuisine.

**Day 2: Architectural Marvels**

- **Morning:** Private tour of the Sagrada Família with a personal art historian guide. Gain exclusive access to areas not open to the public.

- **Lunch:** Dine at Roca Moo, a Michelin-starred restaurant, for a gastronomic experience curated by the Roca brothers.

- **Afternoon:** Explore Park Güell with a private guide. Enjoy a leisurely stroll through Gaudí's colorful mosaic wonderland.

- **Evening:** Relax at the hotel spa followed by a rooftop cocktail at Terrat with panoramic city views.

**Day 3: The Essence of Catalonia**

- **Morning:** Begin with a private helicopter tour over Barcelona and the Catalan countryside, capturing breathtaking views of the region.

- **Lunch:** Enjoy a long, leisurely lunch at Can Roca, the offshoot of the famed El Celler de Can Roca, offering an unforgettable a la carte menu.

- **Afternoon:** Explore the vibrant neighborhood of El Born, visiting the Picasso Museum and boutique shopping with a personal stylist.

- **Dinner:** Experience local flair at Tickets by the Adrià brothers for a fun take on modern tapas.

**Day 4: Cultured and Cool**

- **Morning:** Exclusive tour of the Palau de la Música Catalana. Gain special access to the Musicians' Balcony and the stage.

- **Lunch:** Head to ABaC Restaurant & Hotel, a 3-Michelin-star spot, known for its innovative cuisine by chef Jordi Cruz.

- **Afternoon:** Enjoy a private yacht excursion from Port Vell. Engage with a sommelier for a wine tasting at sea.

- **Evening:** Private Flamenco Show with a gourmet dinner at Tablao Flamenco Cordobés. Enjoy the drama and passion of Spanish dance.

**Day 5: Departure**

- **Morning:** Spend leisure time shopping or at a private art viewing at Fundació Joan Miró.

- **Late Lunch:** Indulge in a farewell lunch at Moments, located at the hotel, offering exquisite local flavors in an elegant setting.

- **Transportation:** Arrange VIP chauffeured service back to the airport.

**Transportation within the City:**
- Utilize a private chauffeur service with a luxury car at your disposal for convenient and comfortable city exploration.

**Estimated Costs:**
- **Flights:** Approximately $15,000 for roundtrip first-class tickets
- **Accommodation:** $6,000 for 5 nights at the Mandarin Oriental
- **Dining:** $4,000 for Michelin-starred meals and private dining experiences
- **Activities:** $3,500 for exclusive tours, private guides, and experiences
- **Transportation:** $2,000 for private luxury car service

All experiences and services are tailored for discerning travelers who expect the best, creating a truly memorable and opulent Barcelona adventure.

**5-Day Barcelona Itinerary for a Couple in Their 30s**

**Budget Overview:**  
- Total Budget: $3,000  
- Prioritize spending on one-of-a-kind experiences and eat at local hotspots.
- Save on accommodation by choosing comfortable, well-located yet moderately priced options.

**Day 1: Arrival and Exploration**  
- **Flight Recommendations:**
  - Return flights from major U.S. cities to Barcelona typically range from $500 to $700 per person.
  - Recommended: Direct flight option with a major airline like Delta or Iberia. Estimated cost: $1,200 for two.

- **Accommodation:**
  - Stay at Hotel Rec Barcelona - Adults Only. Centrally located in El Born district, known for having contemporary design and a rooftop terrace. Approximately $150 per night. Total for 4 nights: $600.

- **Activities:**
  - Afternoon: Explore the Gothic Quarter. Enjoy a stroll through its narrow streets, admire Barcelona Cathedral, and relax at Plaça Reial.
  - Evening: Dinner at Vinitus, known for great tapas. Estimated cost: $60.

- **Transportation:**
  - Use Aerobus from airport to city center. Cost: $7 per person.

**Day 2: Art and Architecture**  
- **Activities:**
  - Morning: Visit La Sagrada Familia. Buy skip-the-line tickets online ($26 per person).
  - Afternoon: Explore Park Güell (buy tickets online for $14 per person).
  - Evening: Dinner at La Pepita for a creative tapas experience. Estimated cost: $70.

- **Transportation:** 
  - Use public transport for the day. Buy a T10 card ($12) for 10 journeys.

**Day 3: Cultural Immersion**  
- **Activities:**
  - Morning: Visit the Picasso Museum. Entry tickets $14 per person.
  - Afternoon: Stroll through El Born to visit local shops and historical sites.
  - Evening: Attend a Flamenco show at Palau de la Música Catalana. Tickets $45 per person. 

- **Dining:**
  - Lunch at Mercado de La Boqueria for a vibrant market experience. Estimated cost: $30.
  - Dinner at La Fonda for authentic paella. Estimated cost: $80.

**Day 4: Beach and Relaxation**  
- **Activities:**
  - Morning: Relax at Barceloneta Beach or rent bikes ($6 each for the day) and ride along the beach promenade.
  - Afternoon: Visit the Barcelona Aquarium. Entry cost $21 per person.
  - Evening: Enjoy sunset cocktails at a beachside bar. Estimated cost: $30.

- **Dining:**
  - Lunch at Makamaka Burger Bar. Estimated cost: $40.
  - Dinner at El Nacional for a modernist dining experience in a stunning space. Estimated cost: $100.

**Day 5: Departure and Souvenir Shopping**  
- **Activities:**
  - Morning: Last-minute shopping on Passeig de Gràcia for souvenirs.
  - Afternoon: Head back to the hotel to check out and relax before departing.
  - Late lunch at Quimet & Quimet, a beloved local tapas bar. Estimated cost: $50.

- **Transportation:**
  - Aerobus back to airport. Cost: $7 per person.

**Estimated Total Costs:**
- Flights: $1,200
- Accommodation: $600
- Meals: $510
- Attractions and Activities: $314
- Local Transport: $44
- Total: Approximately $2,668

This itinerary balances experiences in food, architecture, and cultural immersion while staying within the $3,000 budget, leaving a little flexibility for unexpected expenses or additional splurges.

---

## The Importance of Task Specifications in AI Agents

Task specifications in Agents are crucial for defining what work needs to be done, providing necessary context, and establishing clear expectations for output. Well-crafted task descriptions directly influence the quality, relevance, and usefulness of the results.

### Key Elements of Task Specifications

1. **Description**: The detailed explanation of what the agent needs to accomplish
2. **Expected Output**: A clear statement of what the task should produce
3. **Context**: Additional information or previous task results that provide background
4. **Agent Assignment**: Which agent is responsible for executing the task

### How Task Specifications Impact Results

- **Specificity**: The level of detail in a task description determines how focused the agent's work will be
- **Constraints**: Task descriptions can establish boundaries or requirements that guide the agent
- **Sequential Dependencies**: Tasks can build on each other using context from previous tasks
- **Evaluation Criteria**: The expected output provides a standard against which to judge success


### Key Tradeoffs in Task Specification Design

### Vague Task Specifications
```python
vague_task = Task(
    description="""
    Recommend some good restaurants in New York City.
    """,
    expected_output="A list of restaurant recommendations.",
    agent=food_expert
)
```

**Tradeoffs with Vague Tasks:**
- **Pros:**
  - Quick to write and implement
  - Gives the agent more freedom to interpret and apply its knowledge
  - May discover unexpected recommendations the requester hadn't considered

- **Cons:**
  - Results are likely to be generic and not tailored to specific needs
  - May require multiple iterations to refine toward what's actually needed
  - Agent must make assumptions about unstated preferences
  - Often produces overwhelming lists with too many options
  - Lacks actionable details for decision-making

### Specific Task Specifications
```python
specific_task = Task(
    description="""
    Recommend 3-5 Italian restaurants in Manhattan's West Village neighborhood that:
    
    1. Are suitable for a romantic anniversary dinner
    2. Have a price range of $50-100 per person
    3. Offer vegetarian options (one diner is vegetarian)
    4. Have availability for a party of two this Saturday at 7:00 PM
    5. Have a quiet atmosphere conducive to conversation
    
    For each recommendation, include:
    - Restaurant name and address
    - Price range indicator
    - 1-2 signature dishes to consider
    - Brief explanation of why this restaurant is a good fit
    - Any special notes about ambiance or service
    
    Prioritize establishments with interesting history or exceptional chef credentials.
    """,
    expected_output="A curated list of 3-5 Italian restaurants in Manhattan's West Village that meet the specified criteria, with detailed information about each recommendation.",
    agent=food_expert
)
```

**Tradeoffs with Specific Tasks:**
- **Pros:**
  - Delivers precisely targeted results that meet exact requirements
  - Includes actionable details that facilitate decision-making
  - Reduces the need for follow-up clarification
  - Sets clear evaluation criteria for success
  - Leads to more structured, consistent output

- **Cons:**
  - Takes more time and thought to create
  - May constrain the agent's creativity and limit unexpected discoveries
  - Requires the requester to know their needs in detail upfront
  - Can lead to overfitting if constraints are too narrow
  - May miss valuable options that don't fit strict criteria

### Best Practices

1. **Be as specific as the situation requires** - add more detail when the stakes or costs of getting it wrong are high
2. **Include both constraints and objectives** - what must be true and what would be ideal
3. **Structure the expected format** - outline how the information should be organized
4. **Provide relevant context** - include background information that helps the agent understand the bigger picture
5. **Set clear evaluation criteria** - define how success will be measured

Well-crafted task specifications function as clear contracts between the requester and the agent, ensuring that everyone has the same understanding of what work needs to be done and how success will be measured.

The following code example demonstrates how task specificity impacts the outcome of the agent

In [11]:
llm = ChatOpenAI(model="gpt-4o")

# Create a food expert agent
food_expert = Agent(
    role="Restaurant Recommendation Specialist",
    goal="Provide personalized restaurant recommendations that match the diner's preferences and needs.",
    backstory="You are a food critic with 15 years of experience and extensive knowledge of global cuisines and dining establishments.",
    llm=llm,
    verbose=False
)

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; VAGUE TASK SPECIFICATION</h2></hr></div>'))

# Vague task specification
vague_task = Task(
    description="""
    Recommend some good restaurants in New York City.
    """,
    expected_output="A list of restaurant recommendations.",
    agent=food_expert
)

# Execute vague task
vague_result = food_expert.execute_task(vague_task)
display(Markdown(vague_result))

display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>🔽 &nbsp; SPECIFIC TASK SPECIFICATION</h2></hr></div>'))

# Specific task specification
specific_task = Task(
    description="""
    Recommend 3-5 Italian restaurants in Manhattan's West Village neighborhood that:
    
    1. Are suitable for a romantic anniversary dinner
    2. Have a price range of $50-100 per person
    3. Offer vegetarian options (one diner is vegetarian)
    4. Have availability for a party of two this Saturday at 7:00 PM
    5. Have a quiet atmosphere conducive to conversation
    
    For each recommendation, include:
    - Restaurant name and address
    - Price range indicator
    - 1-2 signature dishes to consider
    - Brief explanation of why this restaurant is a good fit
    - Any special notes about ambiance or service
    
    Prioritize establishments with interesting history or exceptional chef credentials.
    """,
    expected_output="A curated list of 3-5 Italian restaurants in Manhattan's West Village that meet the specified criteria, with detailed information about each recommendation.",
    agent=food_expert
)

# Execute specific task
specific_result = food_expert.execute_task(specific_task)
display(Markdown(specific_result))


display(HTML('<div style="background-color: #000; padding: 10px; border-radius: 5px; border: 1px solid #d3d3d3;"></hr><h2>✅&nbsp; COMPARISON OF RESULTS</h2></hr></div>'))
display(Markdown("The vague task produced a generic list of restaurants without targeted criteria."))
display(Markdown("The specific task produced recommendations tailored to exact needs with actionable details."))

Here are some excellent restaurant recommendations in New York City that should cater to a variety of tastes and occasions:

1. **Eleven Madison Park** - Located at 11 Madison Avenue, it is renowned for its innovative American cuisine and luxurious ambiance. Ideal for special occasions.

2. **Le Bernardin** - Situated at 155 W 51st St, it offers exquisite French seafood and a refined dining experience. Perfect for lovers of fine dining.

3. **Katz's Delicatessen** - Found at 205 E Houston St, this iconic deli is famous for its pastrami sandwiches and a must-visit for a classic NYC dining experience.

4. **Rao's** - Located at 455 E 114th St, it is highly sought-after for its traditional Italian-American dishes and exclusive, intimate setting.

5. **Momofuku Noodle Bar** - Situated at 171 1st Avenue, offering inventive Asian dishes and known for its ramen and pork buns, it provides a casual and hip atmosphere.

6. **The Modern** - At 9 W 53rd St inside the Museum of Modern Art, this restaurant serves contemporary American cuisine with views of the MoMA's sculpture garden.

7. **Peter Luger Steak House** - Located at 178 Broadway in Brooklyn, this steakhouse is legendary for its mouthwatering steaks and classic atmosphere.

8. **Russ & Daughters Café** - Found at 127 Orchard St, it is celebrated for its smoked fish, bagels, and traditional Jewish appetizers in a modern café setting.

9. **Cosme** - Situated at 35 E 21st St, it offers innovative Mexican cuisine and ranked among the World's 50 Best Restaurants: ideal for those seeking culinary creativity.

10. **Carbone** - Located at 181 Thompson St, celebrated for its upscale twists on classic Italian-American fare in a lively and nostalgic setting.

These recommendations capture the culinary diversity and distinct dining experiences offered in New York City, from high-end Michelin-starred establishments to iconic local eateries.

Here are my curated recommendations for Italian restaurants in Manhattan's West Village that are perfect for a romantic anniversary dinner while meeting all your specified criteria:

1. **I Sodi**
   - **Address:** 105 Christopher St, New York, NY 10014
   - **Price Range Indicator:** $$$ (approximately $70-90 per person)
   - **Signature Dishes:** Try the "Spinach Lasagna", known for its delicate texture, and the "Rigatoni Cacio e Pepe", a delightful choice for pasta lovers, including vegetarians.
   - **Why it's a Good Fit:** I Sodi is renowned for its authentic Tuscan flavors and elegantly simple menu. Chef Rita Sodi has garnered acclaim for her focus on traditional, high-quality ingredients and cooking methods. 
   - **Special Notes:** The ambiance is intimate and serene, with a modern yet cozy decor. It features a small number of tables, which ensures a quiet and intimate dining experience.

2. **L'Artusi**
   - **Address:** 228 W 10th St, New York, NY 10014
   - **Price Range Indicator:** $$$ (approximately $60-100 per person)
   - **Signature Dishes:** The "Mushroom Ragu", an exquisite vegetarian option, and "Garganelli with rabbit", for something unique and flavorful.
   - **Why it's a Good Fit:** L'Artusi, a West Village staple, offers a contemporary spin on Italian cuisine. Its focus on seasonal ingredients and expertly crafted wine list enhance the dining experience.
   - **Special Notes:** It has a sophisticated yet lively ambiance with attentive service. The second-floor seating offers a quieter setting perfect for your romantic evening.

3. **Via Carota**
   - **Address:** 51 Grove St, New York, NY 10014
   - **Price Range Indicator:** $$$ (approximately $50-80 per person)
   - **Signature Dishes:** "Ceci Basti", a chickpea stew perfect for vegetarians, and "Tonnarelli Cacio e Pepe", a classic Roman pasta dish.
   - **Why it's a Good Fit:** Co-owned by well-respected chefs Jody Williams and Rita Sodi, Via Carota is celebrated for its rustic, homestyle Italian dishes and impressive culinary authenticity.
   - **Special Notes:** The ambiance channels an Italian trattoria with a cozy, rustic charm, creating a warm and romantic setting. Be sure to confirm the reservation due to its popularity.

4. **Carbone**
   - **Address:** 181 Thompson St, New York, NY 10012
   - **Price Range Indicator:** $$$ (approximately $80-100 per person)
   - **Signature Dishes:** The restaurant's classic "Spicy Rigatoni Vodka" and "Eggplant Parmesan" are must-tries.
   - **Why it's a Good Fit:** Known for its upscale red-sauce Italian-American menu, Carbone is run by the Major Food Group's renowned chefs. Its polished, vintage New York City vibe provides an interesting dining experience.
   - **Special Notes:** The elegant, dimly lit space harks back to mid-century New York glamor, offering an atmosphere conducive to conversation and a memorable anniversary celebration.

5. **Malatesta Trattoria**
   - **Address:** 649 Washington St, New York, NY 10014
   - **Price Range Indicator:** $$ (approximately $50-70 per person)
   - **Signature Dishes:** The vegetarian-friendly "Gnocchi with Tomato and Basil" and the "Osso Buco" for those who enjoy a classic comfort dish.
   - **Why it's a Good Fit:** Known for its cozy and welcoming environment, Malatesta offers traditional Italian charm with a menu that caters well to both vegetarians and meat-eaters alike.
   - **Special Notes:** The atmosphere is quiet and laid-back, ideal for a romantic dinner. The service is noted for being friendly and accommodating, contributing to a pleasant, intimate dining experience.

Ensure to make reservations soon, as these popular spots can fill up quickly, especially for Saturday evenings. Enjoy your anniversary celebration!

The vague task produced a generic list of restaurants without targeted criteria.

The specific task produced recommendations tailored to exact needs with actionable details.

---

## Objective and Task without a framework

Not all AI agent frameworks explicitly support Goal parameters and Task specifications like CrewAI does. When working with different frameworks or directly with model provider APIs, you'll need to adapt how you specify objectives and tasks. Here's how this typically works:

### Direct API Integration

When using model APIs directly (like OpenAI's API), objectives and tasks are simply included as part of the system prompt. For example:

In [4]:
from openai import OpenAI

llm = OpenAI()

role="You are a helpful and truthful assistant who helps user's in finding good restaurants"
objective="Your objective is to find the best Italian restaurants in New York City"
task="Get a list of at least 5 Michelin Star Italian restaurant."

system_prompt=f"""
{role}

{objective}

{task}
"""

response = llm.chat.completions.create(model='gpt-4o',
                                       messages=[
                                           {"role": "system", "content": system_prompt},
                                           {"role": "user", "content": "I am looking for a place for dinner tonight."},
                                       ])
display(Markdown(response.choices[0].message.content))

If you're interested in experiencing some of the finest Italian cuisine in New York City, I recommend considering these Michelin Star Italian restaurants:

1. **Carbone** - Known for its classic Italian-American dishes served in a lively, retro-glam setting. 

2. **Marea** - Renowned for its exceptional seafood and house-made pasta. The ambiance is upscale and perfect for a special evening out.

3. **Ai Fiori** - Offers a sophisticated dining experience with refined Italian and French Riviera-inspired cuisine.

4. **Del Posto** - Known for its luxurious setting and exquisitely prepared Italian dishes.

5. **L’Artusi** - Offers a modern twist on traditional Italian fare with an emphasis on seasonal ingredients.

These restaurants frequently receive high praise for their culinary excellence and would likely provide an unforgettable dining experience. Make sure to check availability, as these popular spots may require reservations in advance.

### Framework-Specific Implementation
Different frameworks handle objectives and tasks in their own ways:

- LangGraph: Provides specialized nodes for injecting instructions at different points in the agent workflow
- LangChain: Uses prompt templates with variables for objectives and tasks
- AutoGen: Uses conversation initialization parameters for agent instruction
- Custom Frameworks: May have proprietary methods for defining agent objectives

### Understanding the Prompt Structure
Regardless of the framework, objectives and tasks are ultimately just parts of the overall prompt. While there's no universal standard, most effective prompts follow this general structure:

- Identity/Persona: Who the agent is
- Objective/Goal: What the agent should accomplish
- Task Specifications: How the agent should proceed
- Other Instructions: Additional context or constraints

<img src="./prompt.png" style="width: 40%" />

The diagram you referenced illustrates this structure well, showing how these components work together to form a complete prompt.

### Best Practices Across Frameworks
Whether you're using a specialized framework or direct API access, these principles apply:

- Be explicit about both what to do (objective) and how to do it (tasks)
- Maintain consistent structure even when terminology differs
- Test different formulations to find what works best for your use case
- Document your approach for future reference and iteration

Remember that the underlying LLM doesn't inherently understand the distinction between "objectives" and "tasks" - these are human-created abstractions to help us organize our instructions effectively.


---

## How do I know if my Objective and Task specifications are good enough? 

### _Meta Prompting_

Determining the quality of your Objective and Task specifications often requires experimentation, which can be time-consuming. This is where meta prompting becomes valuable.
Meta prompting is a technique where you leverage the LLM itself to craft well-formed Objective statements and Task specifications for your agents. Instead of writing these components directly, you:

- Create a META_PROMPT that provides guidance about what you're trying to accomplish
- Include details about your specific project requirements
- Ask the LLM to generate optimized Objective and Task specifications

This approach allows you to benefit from the LLM's understanding of effective prompt design while tailoring the output to your specific needs.
For our Travel Agent example, we can adopt the meta prompting approach [suggested by OpenAI](https://platform.openai.com/docs/guides/prompt-generation) (with minor modifications) to generate effective Objective statements and Task specifications. Here's how it would work:

In [ ]:

from openai import OpenAI

client = OpenAI()

META_PROMPT = """
Given a task description or existing prompt, produce a detailed system prompt to guide a language model in completing the task effectively.

# Guidelines

- Understand the Task: Grasp the main objective, goals, requirements, constraints, and expected output.
- Minimal Changes: If an existing prompt is provided, improve it only if it's simple. For complex prompts, enhance clarity and add missing elements without altering the original structure.
- Reasoning Before Conclusions**: Encourage reasoning steps before any conclusions are reached. ATTENTION! If the user provides examples where the reasoning happens afterward, REVERSE the order! NEVER START EXAMPLES WITH CONCLUSIONS!
    - Reasoning Order: Call out reasoning portions of the prompt and conclusion parts (specific fields by name). For each, determine the ORDER in which this is done, and whether it needs to be reversed.
    - Conclusion, classifications, or results should ALWAYS appear last.
- Clarity and Conciseness: Use clear, specific language. Avoid unnecessary instructions or bland statements.
- Formatting: Use markdown features for readability. DO NOT USE ``` CODE BLOCKS UNLESS SPECIFICALLY REQUESTED.
- Preserve User Content: If the input task or prompt includes extensive guidelines or examples, preserve them entirely, or as closely as possible. If they are vague, consider breaking down into sub-steps. Keep any details, guidelines, examples, variables, or placeholders provided by the user.
- Constants: DO include constants in the prompt, as they are not susceptible to prompt injection. Such as guides, rubrics, and examples.
- Output Format: Explicitly the most appropriate output format, in detail. This should include length and syntax (e.g. short sentence, paragraph, JSON, etc.)
    - For tasks outputting well-defined or structured data (classification, JSON, etc.) bias toward outputting a JSON.
    - JSON should never be wrapped in code blocks (```) unless explicitly requested.

The final prompt you output should adhere to the following structure below. Do not include any additional commentary, only output the completed system prompt. SPECIFICALLY, do not include any additional messages at the start or end of the prompt. (e.g. no "---")

# Output Format

```json
{
    "persona": [The agent's persona; e.g. A seasoned project manager with 10 years of experience...],
    "objective": [The objective goes here],
    "task_specifications": [Task specifications goes here]
}
```
""".strip()

def generate_prompt(task_or_prompt: str):
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": META_PROMPT,
            },
            {
                "role": "user",
                "content": "Please craft clear Persona, Objective statement and Task specifications that can be used with an AI agent for the following:\n" + task_or_prompt,
            },
        ],
    )

    return completion.choices[0].message.content

response = generate_prompt("Find best Michelin star restaurants in New York.")
display(Markdown(response))

```json
{
    "persona": "A culinary enthusiast and experienced food critic with an in-depth knowledge of the New York dining scene.",
    "objective": "Identify and provide a curated list of the best Michelin star restaurants in New York City based on reviews, ratings, and dining experience.",
    "task_specifications": [
        "Research the latest list of Michelin star restaurants in New York City.",
        "Consider factors such as the star rating, chef's reputation, cuisine type, and customer reviews.",
        "Focus on restaurants with exceptional food quality, ambiance, and service as highlighted in critical reviews.",
        "Provide a list of at least 5 top Michelin star restaurants with a brief description of each, including their star rating and location.",
        "Ensure the information is up-to-date and includes any recent changes in restaurant status."
    ]
}
```

### Homework

Try to use Meta Prompting technique to create a persona, objective, and task specification and then use it in a CrewAI agent to complete the task below.

- Book a 3 days travel to Barcelona with leisure and sightseeing in mind
- Book a short-term rental apartment for the stay.